In [113]:
from sklearn import svm
import numpy as np
import os
import numpy
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import collections
def SVM(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)

# Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
#     print(collections.Counter(ArousalFile))
#     print(collections.Counter(ValenceFile))
#     print(np.mean(ValenceFile))
#     print(np.mean(ArousalFile))
    
# SVM Classifier Valence
    clf = SVC(kernel = 'poly')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
#     print(cm)
#     print("Accuracy score of Valence ")
#     print(accuracy_score(y_test, y_predict)*100)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    	
# SVM Classifier Arousal
    clf = SVC(kernel = 'poly')
    clf.fit(X_train, z_train)
    z_predict = clf.predict(X_test)
    cm = confusion_matrix(z_test, z_predict)
#     print(z_test)
#     print(z_predict)
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100]

    
# print("$$$$$$$Arousal$$$$$$$")

def SVMCombined(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceAndArousalEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
    clf = SVC(kernel = 'rbf')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100]



def SVMHS(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'HappyAndSadEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'HS'+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    while ((np.sum(y_train == 0) == 0) or (np.sum(y_train == 1) == 0)):
        print("here")
        X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
        
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
    clf = SVC(kernel = 'rbf')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100]

def Run100(UserName,Window=256,StartTime=0,EndTime=63):
    arousal=[]
    valence=[]
    for i in range (100):
        svmpca=SVM(UserName,Window,StartTime,EndTime)
        arousal+=[svmpca[0]]
        valence+=[svmpca[1]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
    return [np.mean(arousal),np.mean(valence)];
    

    
def RunCombined100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    for i in range (100):
        svm=SVMCombined(UserName,Window,StartTime,EndTime)
        Combined+=svm
    #print("Combined "+str(np.mean(Combined)))
    print(collections.Counter(Combined))
    return np.mean(Combined)
    
def RunHS100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    for i in range (100):
        svm=SVMHS(UserName,Window,StartTime,EndTime)
        Combined+=svm
    #print("Happy and Sad "+str(np.mean(Combined)))
    print(collections.Counter(Combined))
    return np.mean(Combined);

    
if __name__ == "__main__":
    HS=0;
    Combined=0;
    Valence=0
    Arousal=0;
    for i in range(1,10):
        print("s0"+str(i))
        #Happy and Sad
        var =RunHS100("s0"+str(i))
        print("Happy Sad")
        print(var)
        HS+=var
        #All 4 Emotions
        var1 =RunCombined100("s0"+str(i))
        print("Combined Emotions")
        print(var1)
        Combined+=var1
        #Seperated Valence and Arousal
        var2=Run100("s0"+str(i))
        print("Arousal Valence")
        print(var1)
        Valence+=var2[0]
        Arousal+=var2[1]
    for i in range(10,31):
        print("s"+str(i))
        #Happy and Sad
        var =RunHS100("s"+str(i))
        print("Happy Sad")
        print(var)
        HS+=var
        #All 4 Emotions
        var1 =RunCombined100("s"+str(i))
        print("Combined Emotions")
        print(var1)
        Combined+=var1
        #Seperated Valence and Arousal
        var2=Run100("s"+str(i))
        print("Arousal Valence")
        print(var1)
        Valence+=var2[0]
        Arousal+=var2[1]
    print("Average happy sad for first 30 "+str(HS/30))
    print("Average Combined sad for first 30 "+str(Combined/30))
    print("Average Valence for first 30 "+str(Valence/30))
    print("Average Arousal for first 30 "+str(Arousal/30))

s01
Counter({40.0: 41, 60.0: 37, 80.0: 10, 20.0: 9, 0.0: 2, 100.0: 1})
Happy Sad
49.4
Counter({25.0: 35, 37.5: 24, 50.0: 20, 12.5: 16, 0.0: 3, 62.5: 2})
Combined Emotions
31.0
Arousal 61.75
Counter({62.5: 29, 50.0: 25, 75.0: 25, 37.5: 9, 87.5: 6, 100.0: 3, 25.0: 3})
Valence 37.875
Counter({37.5: 55, 25.0: 21, 50.0: 20, 62.5: 3, 12.5: 1})
Arousal Valence
31.0
s02
Counter({80.0: 47, 100.0: 37, 60.0: 13, 40.0: 3})
Happy Sad
83.6
Counter({50.0: 32, 62.5: 22, 37.5: 21, 25.0: 16, 75.0: 4, 12.5: 3, 87.5: 2})
Combined Emotions
46.75
Arousal 62.625
Counter({62.5: 36, 75.0: 29, 50.0: 20, 37.5: 7, 87.5: 5, 100.0: 1, 25.0: 1, 12.5: 1})
Valence 71.75
Counter({75.0: 31, 87.5: 27, 62.5: 21, 50.0: 16, 100.0: 3, 37.5: 2})
Arousal Valence
46.75
s03
Counter({33.33333333333333: 54, 0.0: 26, 66.66666666666666: 20})
Happy Sad
31.33333333333333
Counter({62.5: 35, 50.0: 32, 75.0: 16, 37.5: 7, 87.5: 6, 25.0: 3, 100.0: 1})
Combined Emotions
59.5
Arousal 76.0
Counter({75.0: 36, 87.5: 33, 62.5: 21, 50.0: 6, 100.0

Counter({50.0: 39, 37.5: 24, 62.5: 20, 25.0: 9, 12.5: 3, 75.0: 2, 87.5: 2, 0.0: 1})
Combined Emotions
46.875
Arousal 73.5
Counter({75.0: 37, 87.5: 23, 62.5: 22, 50.0: 7, 100.0: 7, 37.5: 3, 25.0: 1})
Valence 46.0
Counter({50.0: 35, 62.5: 23, 25.0: 18, 37.5: 16, 12.5: 4, 75.0: 3, 87.5: 1})
Arousal Valence
46.875
s23
Counter({60.0: 41, 80.0: 31, 40.0: 14, 100.0: 8, 20.0: 5, 0.0: 1})
Happy Sad
64.0
Counter({37.5: 37, 25.0: 30, 12.5: 22, 50.0: 8, 62.5: 2, 0.0: 1})
Combined Emotions
29.375
Arousal 56.0
Counter({62.5: 35, 50.0: 29, 75.0: 18, 37.5: 11, 12.5: 3, 25.0: 3, 87.5: 1})
Valence 75.375
Counter({75.0: 30, 87.5: 28, 62.5: 24, 100.0: 9, 50.0: 8, 37.5: 1})
Arousal Valence
29.375
s24
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
Counter({100.0: 100})
Happy Sad
100.0
Counter({50.0: 29, 37.5: 24, 25.0: 20, 62.5: 16, 75.0: 6, 12.5: 5})
Combi